In [53]:
import random
import pandas as pd
import yaml
import joblib
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from modeling import WithDropout
from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt

team_rb_stats =  pd.read_csv('data/team_rb_stats.csv',index_col=0)
team_qb_stats= pd.read_csv('data/team_qb_stats.csv',index_col=0)
team_receiver_stats = pd.read_csv('data/team_receiver_stats.csv',index_col=0)
team_stats = pd.read_csv('data/agg/team_stats.csv',index_col=0).rename({'recent_team':'team_name'},axis=1)
opp_stats = pd.read_csv('data/agg/opp_stats.csv',index_col=0).rename({'opponent_team':'team_name'},axis=1)

In [54]:
players = pd.read_parquet('data/agg/player_weekly_agg.parquet')


#players = player_stats.merge(players,left_on=['player_id','week','season'],right_on=['gsis_id','week','season'])

In [55]:
players.loc[(players.player_id=='00-0036322') & (players.season==2024)].iloc[-3]
# will ship '00-0039746'

season                                                                           2024
team_name                                                                         MIN
week                                                                             10.0
game_type                                                                         REG
depth_team                                                                        1.0
last_name                                                                   Jefferson
first_name                                                                     Justin
football_name                                                                  Justin
formation                                                                     Offense
gsis_id                                                                    00-0036322
jersey_number                                                                    18.0
position                                              

In [71]:
model_path = "models/run_yards_gained.pt"
run_yards_model = WithDropout(n_in=14, n_hidden=1000, n_out=130)
run_yards_model.load_state_dict(torch.load(model_path, weights_only=True))

air_yards_path = "models/air_yards.pt"
air_yards_model = WithDropout(n_in=24,n_out=119)
air_yards_model.load_state_dict(torch.load(air_yards_path, weights_only=True))


yac_model_path = "models/yac.pt"
yac_model = WithDropout(n_in=25,n_out=125)
yac_model.load_state_dict(torch.load(yac_model_path, weights_only=True))

<All keys matched successfully>

In [72]:
stat_cols = ['completions', 'attempts',
       'passing_yards', 'passing_tds', 'interceptions', 'sacks', 'sack_yards',
       'sack_fumbles', 'sack_fumbles_lost', 'passing_air_yards',
       'passing_yards_after_catch', 'passing_first_downs', 'passing_epa',
       'passing_2pt_conversions', 'pacr', 'dakota', 'carries', 'rushing_yards',
       'rushing_tds', 'rushing_fumbles', 'rushing_fumbles_lost',
       'rushing_first_downs', 'rushing_epa', 'rushing_2pt_conversions',
       'receptions', 'targets', 'receiving_yards', 'receiving_tds',
       'receiving_fumbles', 'receiving_fumbles_lost', 'receiving_air_yards',
       'receiving_yards_after_catch', 'receiving_first_downs', 'receiving_epa',
       'receiving_2pt_conversions', 'racr', 'target_share', 'air_yards_share',
       'wopr', 'special_teams_tds', 'fantasy_points', 'fantasy_points_ppr']

In [73]:
class Player():
    def __init__(self, d):
        self.name = d["full_name"]
        self.id = d["gsis_id"]
        self.depth_team= int(d['dense_depth'])
        self.stats = {x:0 for x in stat_cols}
        self.stats['air_yards'] = 0
        self.stats['yac'] = 0
        self.features= d.to_dict()

    def show_headshot(self):
        url = self.headshot_url
        response = requests.get(url)
        response.raise_for_status()  # Ensure the request was successful
        img = Image.open(BytesIO(response.content))
        plt.imshow(img)
        plt.axis('off')  # Turn off axis for cleaner display
        plt.show()

    def __getattr__(self, name):
            # Redirect attribute access to the stats dictionary
            if name in self.stats:
                return self.stats[name]
            elif name in self.features:
                return self.features[name]
            raise AttributeError(f"'{self.__class__.__name__}' object has no attribute '{name}'")
    


In [74]:
def fetch_row_or_latest(df, team, season, week):
    df = df.loc[
        (df.team_name == team) & (df.season == season)
    ]
    row = df.loc[
        (df.week == min(df.week.max(), week))
    ].to_dict(orient='records')[0]
    return row

class QB(Player):
    def __init__(self, d):  # noqa: F811
        super().__init__(d)
        self.name = d["full_name"]
        self.features= d.to_dict()

    def __repr__(self):
        return f"QB:{self.name} has {self.completions} completions for {self.passing_yards} yards"


class RB(Player):
    def __init__(self, d):
        super().__init__(d)
        self.name = d["full_name"]
        self.id = d["gsis_id"]
        self.features= d.to_dict()

    def __repr__(self):
        return f"RB:{self.name} has {self.carries} carries for {self.rushing_yards} yards"


class WR(Player):
    def __init__(self, d):
        super().__init__(d)
        self.name = d["full_name"]
        self.id = d["gsis_id"]


    def __repr__(self):
        return f"WR:{self.name} has {self.receptions} receptions for {self.receiving_yards} yards"
    
class TE(Player):
    def __init__(self, d):
        super().__init__(d)
        self.name = d["full_name"]
        self.id = d["gsis_id"]


    def __repr__(self):
        return f"TE:{self.name} has {self.receptions} receptions for {self.receiving_yards} yards"


class Team:
    def __init__(self, name: str, season: int, week: int, use_current_injuries=False):
        self.name = name
        self.score = 0
        self.plays = 0
        self.features = {
            'last_rusher_drive':-1,
            'last_rusher_team':-1
        }
        self.team_stats = fetch_row_or_latest(team_stats, self.name, season, week)
        self.opp_stats = fetch_row_or_latest(opp_stats, self.name, season, week)
        self.players = players.loc[
            (players.team_name == name) & (players.season == season)
        ]
        self.players = self.players.loc[
            (self.players.week == min(self.players.week.max(), week))
            & (self.players.formation == "Offense")
            & (self.players.position.isin(["QB", "WR", "TE", "RB"]))
        ].sort_values(by='dense_depth')

        self.QBs = self.get_players_by_position("QB")
        self.RBs = self.get_players_by_position("RB")
        self.WRs = self.get_players_by_position("WR")
        self.TEs = self.get_players_by_position("TE")
        self.rb_stats = fetch_row_or_latest(team_rb_stats, self.name, season, week)

        self.team_receiver_stats = fetch_row_or_latest(team_receiver_stats, self.name, season, week)

    def get_players_by_position(self, position: str):
        """Filter players by position and create player objects."""
        position_data = self.players[(self.players["position"] == position)].fillna(0)

        # Create player objects based on position
        players = []
        for _, player_data in position_data.iterrows():
            if position == "WR":
                players.append(WR(player_data))
            elif position == "RB":
                players.append(RB(player_data))
            elif position == "QB":
                players.append(QB(player_data))
            elif position == "TE":
                players.append(TE(player_data))
        return players
    
    def get_depth_pos(self, pos: str, depth: int):
        '''input a position and team depth, to get the player
            used to go from ML output -> player object'''
        if pos == 'WR':
            for player in self.WRs:
                if player.depth_team == depth:
                    return player
        if pos == 'RB':
            for player in self.RBs:
                if player.depth_team == depth:
                    return player
        if pos == 'TE':
            for player in self.TEs:
                if player.depth_team == depth:
                    return player
        raise ValueError('You want a player that does not exist')
    


    

    def __repr__(self):
        return f"{self.name} has {self.score} points"

In [75]:
team_stats.columns

Index(['team_name', 'season', 'week', 'completions_team_roll',
       'attempts_team_roll', 'carries_team_roll', 'passing_yards_team_roll',
       'rushing_yards_team_roll', 'pass_pct_team_roll',
       'receiving_yards_team_roll'],
      dtype='object')

In [76]:
vik = Team('MIN',2024, 13)
det = Team('DET',2024, 13)
vik.team_stats

C:\Users\Sharedw\AppData\Local\Temp\ipykernel_28916\4291210146.py:82: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  position_data = self.players[(self.players["position"] == position)].fillna(0)
C:\Users\Sharedw\AppData\Local\Temp\ipykernel_28916\4291210146.py:82: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  position_data = self.players[(self.players["position"] == position)].fillna(0)


{'team_name': 'MIN',
 'season': 2024,
 'week': 12,
 'completions_team_roll': 21.571428571428573,
 'attempts_team_roll': 32.0,
 'carries_team_roll': 29.714285714285715,
 'passing_yards_team_roll': 263.57142857142856,
 'rushing_yards_team_roll': 113.28571428571428,
 'pass_pct_team_roll': 0.5226988013408093,
 'receiving_yards_team_roll': 263.57142857142856}

In [ ]:
class GameState:


	def __init__(self, home, away, config):
		self.home =home
		self.away = away
		self.quarter = 1
		self.possession = None
		self.defending = None
		self.down = 1
		self.ydstogo = 10
		self.ball_position = 65  # Yardline (0-100), 0 is score, 100 is safety
		self.clock = 900  # Seconds in the current quarter (15 mins = 900 seconds)
		self.drive = 0
		self.home_timeouts = 3
		self.away_timeouts = 3
		self.pbp = []
		self.player = None
		self.run_or_pass = joblib.load('models/run_or_pass.joblib')
		self.run_or_pass_cols = config['run_or_pass_cols']
		self.choose_rusher = joblib.load('models/choose_rusher.joblib')
		self.choose_rusher_cols = config['choose_rusher_cols']
		self.choose_receiver = joblib.load('models/choose_receiver.joblib')
		self.choose_receiver_cols = config['choose_receiver_cols']
		self.air_yards_cols = config['air_yards_cols']
		self.receiver_idx_to_pos = config['receiver_idx_to_pos']
		self.rush_yard_cols = config['rush_yard_cols']
		self.complete_pass_cols = config['complete_pass_cols']
		self.complete_pass_model = joblib.load('models/complete_pass.joblib')
		self.play_encoding = config['play_encoding']
		self.wind = random.randint(0,10)
		self.temp = random.randint(40,90)
		self.play_functions = {
			'field_goal': self.field_goal,
			'no_play': self.run_play,
			'pass': self.pass_play,
			'punt': self.punt,
			'qb_kneel': self.qb_kneel,
			'qb_spike': self.qb_spike,
			'run': self.run_play
			}
		
		
	def switch_poss(self):
		self.possession.features['last_rusher_drive'] = -1
		self.possession = self.away if self.possession == self.home else self.home 
		self.defending = self.away if self.defending == self.home else self.home 
		self.ball_position = 100 - min(self.ball_position, 99)
		self.down = 1
		self.ydstogo = min(10, self.ball_position)
		self.drive += 1
		return
	
	def kickoff(self):
		self.switch_poss()
		self.ball_position = 65
		pass
	
	def start_game(self):
		lost_kickoff = random.choice((self.home, self.away))
		self.possession=lost_kickoff
		self.kickoff()
		print(f'{self.possession.name} has won the kickoff')
		self.log_play('kickoff',0)


	def self.collect_features(**args):
		features = {}
		for arg in argv:
			features.update(arg)
		return features

	def play(self, team):
		team.plays += 1
		raw_features = self.collect_features(self.pbp[-1], team.team_stats, self.defending.opp_stats)
		raw_features = {}
		raw_features.update()
		raw_features.update(team.team_stats)
		raw_features.update(self.defending.opp_stats)
		features = [raw_features[key] for key in self.run_or_pass_cols]
		preds = self.run_or_pass.predict_proba([features])
		play_type_int = np.random.choice(len(preds[0]), p=preds[0])
		play_type = self.play_encoding.get(play_type_int, 1)
		yds = self.play_functions[play_type](team)
		self.log_play(play_type, yds)
		self.ydstogo -= yds
		self.ball_position -= yds
		self.clock -= random.randint(15, 40)
		self.td_check(team)
		self.check_downs(team)


	def log_play(self,play_type, yds,verbose=False):
		"""Logs the context of the game state at each play."""
		play_data = {
			"possession": self.possession.name,
			"quarter": self.quarter,
			"down": self.down,
			"ydstogo": self.ydstogo,
			"goal_to_go": int(self.ball_position < 10),
			"yardline_100": self.ball_position,
			"total_home_score": self.home.score,
			"total_away_score": self.away.score,
			"posteam_score": self.possession.score,
			"defteam_score": self.home.score if self.possession == self.away else self.away.score,
			"score_differential": (self.home.score - self.away.score),
			"wind": self.wind,
			"temp": self.temp,
			"quarter_seconds_remaining": self.clock,
			'half_seconds_remaining': self.clock + (900 * (self.quarter%2)),
    		'game_seconds_remaining': self.clock + (900 * (4-self.quarter)),
			'drive':self.drive,
			'spread_line':-2.5,
			'total_line': 42.5,
			'play_type': play_type,
			'yards_gained': yds,
			'player': self.player
		}
		if verbose:
			print(
				f'{self.possession.name} {play_type} for {yds} yards, {self.pbp[-1]['yardline_100']} yd line,' + 
				f' {self.pbp[-1]['ydstogo']} yds to go on {self.pbp[-1]['down']} down.' + 
				f' {self.pbp[-1]['quarter_seconds_remaining'] // 60}:{self.pbp[-1]['quarter_seconds_remaining']  % 60} left'
			)
		self.pbp.append(play_data)


	
	def sample_run_yards(self, model, player):
		#rush_yards_cols = self.rush_yards_cols
		raw_features = dict(self.pbp[-1])
		raw_features.update(player.features)
		x = [raw_features[key] for key in self.rush_yard_cols]
		x = torch.tensor(x)
		with torch.no_grad():
			preds = model(x.reshape(1,-1))[0]
			preds = torch.softmax(preds,0)
		sample = (torch.multinomial(preds,1)).item() - 30
		return min((sample, raw_features['yardline_100']))

	def sample_air_and_yac(self, air_model, yac_model, player):
		raw_features = dict(self.pbp[-1])
		raw_features.update(player.features)
		raw_features.update(self.defending.features)
		x = [raw_features[key] for key in self.air_yards_cols]
		x = torch.tensor(x)
		with torch.no_grad():
			preds = air_model(x.reshape(1,-1))[0]
			preds = torch.softmax(preds,0)
		air_yards = (torch.multinomial(preds,1)).item() - 20
		if air_yards >= self.ball_position: #touchdown at catch
			return air_yards, 0
		
		x = torch.cat((x, torch.tensor([air_yards])))
		with torch.no_grad():
			preds = yac_model(x.reshape(1,-1))[0]
			preds = torch.softmax(preds,0)
		yac = (torch.multinomial(preds,1)).item() - 25
		return air_yards, min(yac, (self.ball_position - air_yards))
	
	def sample_completion(self, qb, receiver, air_yards):
		raw_features = dict(self.pbp[-1])
		raw_features.update(receiver.features)
		raw_features['air_yards'] = air_yards
		qb_features = {(key+'_qb'):value for key,value in qb.features.items()}
		raw_features.update(qb_features)
		features = [raw_features[key] for key in self.complete_pass_cols]
		preds = self.complete_pass_model.predict_proba([features])
		receiver = np.random.choice(len(preds[0]), p=preds[0])
		return  np.random.choice(len(preds[0]), p=preds[0])
		
		

	def run_play(self, team):
		raw_features = dict(self.pbp[-1])
		raw_features.update(team.rb_stats)
		raw_features.update(team.features)
		raw_features.update(self.defending.opp_stats)
		features = [raw_features[key] for key in self.choose_rusher_cols]
		preds = self.choose_rusher.predict_proba([features])
		rusher_depth = np.random.choice(len(preds[0]), p=preds[0])
		try:
			player = team.get_depth_pos('RB', int(rusher_depth))
		except:
			player = team.RBs[0]
		player.carries += 1
		yds = self.sample_run_yards(run_yards_model, player)
		player.rushing_yards += yds
		self.player = player.name
		team.features['last_rusher_team'] = rusher_depth
		team.features['last_rusher_drive'] = rusher_depth
		return yds
	




	def pass_play(self, team):
		passer = team.QBs[0]
		raw_features = dict(self.pbp[-1])
		raw_features.update(team.team_receiver_stats)
		raw_features.update(team.features)
		features = [raw_features[key] for key in self.choose_receiver_cols]
		preds = self.choose_receiver.predict_proba([features])
		receiver = np.random.choice(len(preds[0]), p=preds[0])
		pos, depth = self.receiver_idx_to_pos[receiver].split('_')
		receiver = team.get_depth_pos(pos, int(depth))
		passer.attempts += 1
		receiver.targets += 1
		air_yards, yac = self.sample_air_and_yac(air_yards_model,yac_model, receiver)
		qb_features = {}
		qb_features.update(passer.features)

		if self.sample_completion(passer, receiver, air_yards):
			passer.completions += 1
			receiver.receptions += 1
			yds = air_yards + yac
			receiver.air_yards += air_yards
			receiver.yac += yac
			receiver.receiving_yards += yds
			passer.passing_yards += yds
		else:
			yds = 0
		self.player = receiver.name
		return yds
	
	
	def punt(self, team):
		self.switch_poss()
		self.ball_position += random.randint(30, 60)
		if self.ball_position >= 100:
			self.ball_position = 20
		self.player=None
		return 0
	
	def field_goal(self,team):
		result = random.randint(0,100)
		if result > (2 * self.ball_position):
			team.score += 3
			self.switch_poss()
			self.ball_position = 65
			print(f'{team.name} scored a FG')
		else:
			print(f'{team.name} missed FG')
			self.switch_poss()
		self.player=None
		return 0
	
	def qb_kneel(self, team):
		# Implementation of qb kneel play
		print("QB kneel executed.")
		return -1

	def qb_spike(self, team):
		# Implementation of qb spike play
		print("QB spike executed.")
		return 0
	
	def td_check(self, team):
		if self.ball_position <= 0:
			team.score += 7
			self.kickoff()
			print(f'{team.name} scored a TD')
		return
	
	def check_downs(self, team):
		if self.ydstogo <= 0:
			self.ydstogo = 10
			self.down = 1
		elif self.down == 4:
			self.switch_poss()
		else:
			self.down += 1


	def play_quarter(self):
		self.clock = 900
		while self.clock > 0:
			self.play(self.possession)
		self.quarter += 1
		print(f'{self.home.name}:{self.home.score}')
		print(f'{self.away.name}:{self.away.score}')

	def play_game(self):
		while self.quarter <= 4:
			self.play_quarter()
			print(self.quarter)


SyntaxError: expected '(' (4022779531.py, line 67)

In [124]:
vik.team_stats

{'team_name': 'MIN',
 'season': 2024,
 'week': 11,
 'completions_team_roll': 21.142857142857142,
 'attempts_team_roll': 31.0,
 'carries_team_roll': 30.714285714285715,
 'passing_yards_team_roll': 253.7142857142857,
 'rushing_yards_team_roll': 112.71428571428572,
 'pass_pct_team_roll': 0.5065697690827448,
 'receiving_yards_team_roll': 253.7142857142857}

In [125]:
with open('models/feature_config.yaml', 'r') as file:
    config = yaml.safe_load(file)

vik = Team('MIN',2024, 11)
det = Team('DET',2024, 11)
game = GameState(vik, det, config)
game.start_game()
game.play_game()


DET has won the kickoff


KeyError: 'rushing_yards_team_roll'

In [ ]:
vik.QBs[0].attempts

40

In [121]:
for i in range(10):
    game = GameState(vik, det, config)
    game.start_game()
    game.play_game()
    game.home.score=0
    game.away.score=0

DET has won the kickoff


KeyError: 'passing_yards_opp_roll'

In [122]:
pbp = pd.json_normalize(game.pbp)
pbp

,possession,quarter,down,ydstogo,goal_to_go,yardline_100,total_home_score,total_away_score,posteam_score,defteam_score,...,temp,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,drive,spread_line,total_line,play_type,yards_gained,player
0,DET,1,1,10,0,65,0,0,0,0,...,84,900,1800,3600,1,-2.5,42.5,kickoff,0,None


In [113]:
vik.WRs

[WR:Justin Jefferson has 90 receptions for 1106 yards,
 WR:Jordan Addison has 38 receptions for 362 yards,
 WR:Jalen Nailor has 10 receptions for 134 yards,
 WR:Brandon Powell has 1 receptions for 17 yards,
 WR:Trent Sherfield has 9 receptions for 99 yards]

In [116]:
a = {1:2}
b = {}
b.update(a)
a

{1: 2}

In [117]:
b

{1: 2}